# Lasso
Lasso 實施了L1正規化，使用CPU和GPU的運算差別：

1. 使用龐大的資料集(多列少欄)，在GPU上運算的時間會比在CPU上，來的快許多
2. 使用較小的資料集時，使用GPU的均方誤差(MSE)會比在CPU上運算時小很多


## 1. Import 相關套件

In [12]:
import cudf
import numpy as np
from ncue import make_regression, train_test_split
from ncue.linear_model import Lasso as cuLasso
from ncue.metrics.regression import r2_score
from sklearn.linear_model import Lasso as skLasso

## 2. 定義 Parameters

In [13]:
n_samples = 2**17
n_features = 500

learning_rate = 0.001
algorithm = "cyclic"
random_state=23

## 3. 產生測試資料

In [14]:
%%time
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame.from_gpu_matrix(X)
y = cudf.DataFrame.from_gpu_matrix(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

CPU times: user 1.6 s, sys: 171 ms, total: 1.77 s
Wall time: 1.78 s


In [15]:
# 將資料從GPU MEMORY複製到RAM，方便sklearn使用，以利最後結果的比對
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

## 4. Scikit-learn 模型(CPU)

### 訓練模型

In [16]:
%%time
ols_sk = skLasso(alpha=np.array([learning_rate]),
                 fit_intercept = True,
                 normalize = False,
                 max_iter = 1000,
                 selection=algorithm,
                 tol=1e-10)

ols_sk.fit(X_train, y_train);

CPU times: user 2min 22s, sys: 2min 23s, total: 4min 46s
Wall time: 18.6 s


/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 51.62523651123047, tolerance: 0.035234078764915466
  positive)


### 預測

In [17]:
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 141 ms, sys: 584 ms, total: 724 ms
Wall time: 49 ms


### 驗證

In [18]:
%%time
r2_score_sk = r2_score(y_cudf_test, predict_sk)

CPU times: user 46.6 ms, sys: 250 ms, total: 297 ms
Wall time: 20.5 ms


## 5. NCUE 模型(GPU)

### 訓練模型

In [19]:
%%time
ols_ncue = cuLasso(alpha=np.array([learning_rate]),
                   fit_intercept = True,
                   normalize = False,
                   max_iter = 1000,
                   selection=algorithm,
                   tol=1e-10)

ols_ncue.fit(X_cudf, y_cudf)
#我們的模型支援 array-like objects，e.g. NumPy arrays, cuDF DataFrames

CPU times: user 380 ms, sys: 368 ms, total: 749 ms
Wall time: 294 ms


### 預測

In [20]:
%%time
predict_ncue = ols_ncue.predict(X_cudf_test).to_array()

CPU times: user 228 ms, sys: 4.34 ms, total: 232 ms
Wall time: 232 ms


### 驗證

In [21]:
%%time
r2_score_ncue = r2_score(y_cudf_test, predict_ncue)

CPU times: user 674 µs, sys: 671 µs, total: 1.35 ms
Wall time: 1.04 ms


## 6. 比對運算結果(CPU vs. GPU)

In [22]:
print("R^2 score (CPU):  %s" % r2_score_sk)
print("R^2 score (GPU): %s" % r2_score_ncue)

R^2 score (CPU):  1.0
R^2 score (GPU): 1.0
